In [4]:
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import string
from nltk.corpus import wordnet
import numpy as np
# package to calculate distance
from scipy.spatial import distance

In [1]:
def tokenize(text):
    #token_count = None
    # add your code here
    #convert the string to lowercase
    #text.lower()
    #Each token has at least two characters.
    #The first/last character can only be a letter (i.e. a-z) or a number (0-9)
    #In the middle, there are 0 or more characters, which can only be letters (a-z),
    #numbers (0-9), hyphens ("-"), underscores ("_"), dot ("."), or "@" symbols.

    pattern=r'\w+[\w\-_\.@]*\w+'   
    tokens=nltk.regexp_tokenize(text.lower(), pattern)

    #print(len(tokens))
    #print (tokens)

    wordnet_lemmatizer = WordNetLemmatizer()
    # then find the POS tag of each word
    # tagged_token is a list of (word, pos_tag)
    tagged_tokens= nltk.pos_tag(tokens)
    stop_words = stopwords.words('english')
    def get_wordnet_pos(pos_tag):
        
        # if pos tag starts with 'J'
        if pos_tag.startswith('J'):
            # return wordnet tag "ADJ"
            return wordnet.ADJ
    
        # if pos tag starts with 'V'
        elif pos_tag.startswith('V'):
            # return wordnet tag "VERB"
            return wordnet.VERB
    
        # if pos tag starts with 'N'
        elif pos_tag.startswith('N'):
            # return wordnet tag "NOUN"
            return wordnet.NOUN
    
        elif pos_tag.startswith('R'):
            return wordnet.ADV
        else:
            # be default, return wordnet tag "NOUN"
            return wordnet.NOUN

    lemmatized_words=[wordnet_lemmatizer.lemmatize\
          (word, get_wordnet_pos(tag)) \
          # tagged_tokens is a list of tuples (word, tag)
          for (word, tag) in tagged_tokens \
          # remove stop words
          if word not in stop_words and \
          # remove punctuations
          word not in string.punctuation]
    #print(lemmatized_words)
    token_count=nltk.FreqDist(lemmatized_words)
    #token_count
    return token_count

In [5]:
data=pd.read_csv("qa.csv", header=0)
docs=data["question"].values.tolist()
#  process all documents to get list of token list
docs_tokens={idx:tokenize(doc) \
         for idx,doc in enumerate(docs)}

# get document-term matrix
dtm=pd.DataFrame.from_dict(docs_tokens, orient="index" )
dtm=dtm.fillna(0)

#  get normalized term frequency (tf) matrix        
tf=dtm.values
doc_len=tf.sum(axis=1)
tf=np.divide(tf.T, doc_len).T

# get idf
df=np.where(tf>0,1,0)
#idf=np.log(np.divide(len(docs), \
#    np.sum(df, axis=0)))+1

smoothed_idf=np.log(np.divide(len(docs)+1, np.sum(df, axis=0)+1))+1    
smoothed_tf_idf=tf*smoothed_idf

# calculate cosince distance of every pair of documents 
# convert the distance object into a square matrix form
# similarity is 1-distance
similarity=1-distance.squareform\
(distance.pdist(smoothed_tf_idf, 'cosine'))

In [13]:
similarity

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [11]:
type(docs)

list

In [12]:
type(similarity)

numpy.ndarray